## Import

In [1]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [3]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':100,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':1024,
    'SEED':41,
    'DATA_CNT': 1
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### 데이터 불러오기

In [5]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/2023_Summer_Aimers/데이터'

Mounted at /content/drive


In [6]:
train_data = pd.read_csv(path+'/rawdata/train.csv').drop(columns=['ID','제품'])

### 데이터 전처리

In [7]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[4:]

# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)

# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)

# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [8]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [9]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size

    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])

        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]

    return input_data, target_data

In [10]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)

    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])

        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data

    return input_data

In [47]:
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

In [14]:
def train_val_split(data, input_data, target_data, series_size, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):

    val_index = sorted(random.sample(range(series_size),int(series_size*0.2)))

    num_rows=len(data)

    val_inputs = [] # 일 순서대로 담을 예정
    val_targets = []
    for i in tqdm(val_index):
        inputs = np.empty((num_rows, train_size, input_data.shape[2]))
        targets = np.empty((num_rows, predict_size))
        for j in range(num_rows):
            inputs[j] = input_data[j * series_size + i]
            targets[j] = target_data[j * series_size + i]
        val_inputs.append(inputs)
        val_targets.append(targets)

    train_series_size = series_size - len(val_index)

    train_input = np.empty((num_rows * train_series_size, train_size, input_data.shape[2]))
    train_target = np.empty((num_rows * train_series_size, predict_size))

    # train 데이터 생성
    k = 0
    for i in tqdm(range(series_size)):
        if i not in val_index:
            for j in range(num_rows):
                train_input[k] = input_data[j * series_size + i]
                train_target[k] = target_data[j * series_size + i]
                k += 1

    return train_input, train_target, val_inputs, val_targets

### Custom Dataset

In [48]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])

    def __len__(self):
        return len(self.X)

### 모델 선언

In [13]:
indexs_bigcat={}
for bigcat in train_data['대분류'].unique():
    indexs_bigcat[bigcat] = list(train_data.loc[train_data['대분류']==bigcat].index)

indexs_bigcat.keys()

def PSFA(pred, target):
    PSFA = 1
    print(pred.shape)
    print(target.shape)
    for cat in range(5):
        ids = indexs_bigcat[cat]

        for day in range(21):
            total_sell = np.sum(target[ids, day]) # day별 총 판매량
            pred_values = pred[ids, day] # day별 예측 판매량
            target_values = target[ids, day] # day별 실제 판매량

            # 실제 판매와 예측 판매가 같은 경우 오차가 없는 것으로 간주
            denominator = np.maximum(target_values, pred_values)
            diffs = np.where(denominator!=0, np.abs(target_values - pred_values) / denominator, 0)

            if total_sell != 0:
                sell_weights = target_values / total_sell  # Item별 day 총 판매량 내 비중
            else:
                sell_weights = np.ones_like(target_values) / len(ids)  # 1 / len(ids)로 대체

            if not np.isnan(diffs).any():  # diffs에 NaN이 없는 경우에만 PSFA 값 업데이트
                PSFA -= np.sum(diffs * sell_weights) / (21 * 5)


    return PSFA

In [14]:
import torch
import torch.nn as nn

# early_stopping 객체 선언(3번의 epoch 연속으로 loss 미개선 시에 조기 종료 예시)
class CNN_LSTM_Model(nn.Module):
    def __init__(self, input_size=5, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(CNN_LSTM_Model, self).__init__()
        self.hidden_size = hidden_size

        # CNN layers for feature extraction
        self.cnn = nn.Sequential(
            nn.Conv1d(input_size, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )

        # LSTM layer
        self.lstm = nn.LSTM(128, hidden_size, batch_first=True)

        # Fully connected layer
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )

        self.actv = nn.ReLU()

    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 5)
        batch_size = x.size(0)

        # Transpose to match Conv1d input shape
        x = x.transpose(1, 2)

        # CNN layer
        cnn_out = self.cnn(x)

        # Transpose back to LSTM input shape
        lstm_input = cnn_out.transpose(1, 2)

        # LSTM layer
        lstm_out, _ = self.lstm(lstm_input)

        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))

        return output.squeeze(1)


### 모델 학습

In [15]:
import torch
import numpy as np


class EarlyStopping(object):
    def __init__(self, patience=2, save_path="model.pth"):
        self._min_loss = np.inf
        self._patience = patience
        self._path = save_path
        self.__counter = 0

    def should_stop(self, model, loss):
        if loss < self._min_loss:
            self._min_loss = loss
            self.__counter = 0
            torch.save(model.state_dict(), self._path)
        elif loss > self._min_loss:
            self.__counter += 1
            if self.__counter >= self._patience:
                return True
        return False

    def load(self, model):
        model.load_state_dict(torch.load(self._path))
        return model

    @property
    def counter(self):
        return self.__counter

In [26]:
def validation(model, val_loader, device):
    pred = []
    target = []

    with torch.no_grad():
        for X, Y in iter(val_loader):
            X = X.to(device)
            Y = Y.to(device)

            Y = Y.cpu().numpy()
            target.extend(Y)
            output = model(X)
            output = output.cpu().numpy()
            pred.extend(output)

    pred = np.array(pred)
    target = np.array(target)

    # 추론 결과를 inverse scaling
    for idx in range(len(pred)):
        pred[idx, :] = pred[idx, :] * (scale_max_dict[12712+idx//353] - scale_min_dict[12712+idx//353]) + scale_min_dict[12712+idx//353]
        target[idx, :] = target[idx, :] * (scale_max_dict[12712+idx//353] - scale_min_dict[12712+idx//353]) + scale_min_dict[12712+idx//353]

    # 결과 후처리
    pred = np.round(pred, 0).astype(int)
    target = np.round(target, 0).astype(int)

    return PSFA(pred, target), PSFA(pred, target)

In [27]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    early_stopper = EarlyStopping(patience=3)

    for epoch in range(1, CFG['EPOCHS']+1):
        val_loss, psfa_value = validation(model, val_loader, device)

        train_loss = []
        train_mae = []

        model.train()

        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

            break

        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        if early_stopper.should_stop(model, psfa_value):
            print(f"EarlyStopping: [Epoch: {epoch - early_stopper.counter}]")
            break
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

In [49]:
# Train / Validation Split
## 이거 두번 돌리면 안됩니다.....

data_len = len(train_input)
val_input_ = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input_ = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

train_dataset = CustomDataset(train_input_, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input_, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Run !!

In [50]:
model = CNN_LSTM_Model()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

(1121834, 21)
(1121834, 21)
(1121834, 21)
(1121834, 21)


<ipython-input-13-84f2e60f4323>:21: RuntimeWarning: invalid value encountered in divide
  diffs = np.where(denominator!=0, np.abs(target_values - pred_values) / denominator, 0)


  0%|          | 0/4383 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.03839] Val Loss : [0.19488]
Model Saved
(1121834, 21)
(1121834, 21)
(1121834, 21)
(1121834, 21)


  0%|          | 0/4383 [00:00<?, ?it/s]

...

  0%|          | 0/4383 [00:00<?, ?it/s]

Epoch : [4] Train Loss : [0.03765] Val Loss : [0.13609]
Model Saved
(1121834, 21)
(1121834, 21)
(1121834, 21)
(1121834, 21)


  0%|          | 0/4383 [00:00<?, ?it/s]

Epoch : [5] Train Loss : [0.03408] Val Loss : [0.12934]
Model Saved


KeyboardInterrupt: ignored

In [46]:
len(scale_min_dict), val_target.shape[0]//353, val_input_.shape[0]//353, train_target.shape[0]//353, (train_input_.shape[0]* 0.8)//353 , train_input_.shape[0]//353, train_input_.shape[0]

(15890, 3178, 3178, 9534, 10169.0, 12712, 4487336)

## 모델 추론

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    predictions = []

    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)

            output = model(X)

            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()

            predictions.extend(output)

    return np.array(predictions)

In [ ]:
pred = inference(infer_model, test_loader, device)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

(15890, 21)

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
submit.iloc[:,1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)